# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [6]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [7]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [8]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [9]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [10]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [11]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [12]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [13]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [14]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [15]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [16]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [17]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [18]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [19]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to make sure that the team is effectively monitoring and analyzing market data for AAPL to identify trends and predict market movements. I should delegate tasks and ask questions to ensure we are on track to provide insights and alerts about significant market opportunities or threats for AAPL.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning techniques", "context": "We need detailed insights on market trends and predictions for AAPL based on the data analysis"}

> Entering new CrewAgentExecutor chain...
I need to gather real-time data on AAPL to analyze market trends 

- Display the final result as Markdown.

In [20]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies and Execution Plans:

Introduction:
This comprehensive risk analysis report aims to identify potential risks associated with the approved trading strategies and execution plans for AAPL. The analysis will provide insights into the various risk factors that could impact trading activities and suggest mitigation strategies to minimize potential losses and optimize performance.

Risks Identified:
1. Geopolitical Tensions: The ongoing geopolitical tensions between major economies could impact AAPL's global supply chain and market access, leading to disruptions in production and distribution.
2. Future Product Profitability: Changes in consumer preferences, technological advancements, or regulatory challenges could affect the profitability of future Apple products, impacting stock performance.
3. Stock Valuation: The premium valuation of AAPL stock poses a risk to investors, especially in volatile market conditions where technical weaknesses could lead to significant price fluctuations.
4. Market Volatility: The unpredictable nature of financial markets can expose AAPL to sudden price movements and increased trading risks, requiring proactive risk management strategies.
5. Regulatory Challenges: Legal and regulatory changes in key markets could impact AAPL's operations, sales, and profitability, requiring compliance measures and adaptation to new regulatory environments.
6. Supply Chain Disruptions: Export restrictions, trade disputes, or logistical challenges could disrupt AAPL's supply chain, affecting production schedules, product availability, and revenue streams.
7. Technological Changes: Rapid technological advancements and market innovations could render current trading strategies obsolete, necessitating continuous adaptation and innovation in trading practices.

Mitigation Strategies:
1. Diversification: Spread trading activities across different markets, asset classes, and trading strategies to minimize concentration risk and exposure to specific market events.
2. Risk Monitoring: Implement real-time risk management tools and monitoring systems to track market developments, assess risk exposure, and adjust trading positions accordingly.
3. Contingency Planning: Develop contingency plans for potential risk scenarios, including supply chain disruptions, regulatory changes, and market volatility, to ensure business continuity and risk mitigation.
4. Compliance Measures: Stay abreast of legal and regulatory developments, adhere to compliance requirements, and implement robust governance frameworks to mitigate regulatory risks.
5. Collaboration: Foster partnerships with industry experts, risk advisors, and legal counsel to gain insights into emerging risks, regulatory changes, and best practices in risk management.
6. Technology Adoption: Embrace technological solutions for risk assessment, algorithmic trading, and real-time data analysis to enhance risk management capabilities and optimize trading performance.

Conclusion:
In conclusion, the identified risks associated with AAPL trading strategies and execution plans pose significant challenges to trading activities and investment decisions. By analyzing these risks, developing mitigation strategies, and implementing proactive risk management measures, the firm can navigate market uncertainties, protect investment capital, and achieve sustainable trading success in the dynamic landscape of financial markets.